In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# 📌 Charger le modèle TFLite
TFLITE_MODEL_PATH = "cacao_disease_model.tflite"
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()

# 📌 Obtenir les détails des tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 📌 Dictionnaire contenant les descriptions et causes des maladies
disease_info = {
    "black_pod_rot": {
        "description": """🌿 *Pourriture Noire de la Cabosse (Black Pod Rot)* 🌿  
        La pourriture noire est une **maladie fongique** grave causée par *Phytophthora spp.*, qui attaque les cabosses de cacao.  
        Elle commence par l'apparition de **taches brunes foncées** qui s'élargissent progressivement, jusqu'à **recouvrir toute la cabosse** et la faire pourrir.  
        En l'absence de traitement, elle peut **détruire jusqu'à 40% de la production** d'une plantation.""",
        "causes": """- **Humidité élevée et fortes pluies** favorisant la propagation du champignon.  
        - **Sol mal drainé** qui retient l'eau, créant un environnement idéal pour la maladie.  
        - **Spores fongiques transportées par le vent, l’eau ou les insectes**.  
        - **Absence de pratiques de lutte préventive** comme l'élagage et l'application de fongicides."""
    },
    "pod_borer": {
        "description": """🦋 *Foreur de Cabosse (Pod Borer)* 🦋  
        Le foreur de cabosse (*Conopomorpha cramerella*) est un insecte nuisible dont **les larves pénètrent dans la cabosse** pour se nourrir des graines de cacao.  
        Cette infestation rend les cabosses **difficiles à ouvrir**, et les fèves à l'intérieur deviennent **collantes et de mauvaise qualité**.  
        Ce ravageur est un problème majeur en Asie et en Afrique, réduisant considérablement le rendement des plantations.""",
        "causes": """- **Présence de papillons adultes pondant sur les cabosses**.  
        - **Absence de lutte biologique** (par exemple, pas de prédateurs naturels des larves).  
        - **Manque d’hygiène dans la plantation**, favorisant la prolifération des insectes.  
        - **Conditions climatiques favorables à la reproduction du papillon**."""
    },
    "healthy": {
        "description": """🌱 *Cabosse en Bonne Santé* 🌱  
        Félicitations ! 🎉 Votre cabosse est **saine et en parfait état**.  
        Elle ne présente **aucune maladie ni infestation**.  
        Cela signifie que les **pratiques agricoles mises en place sont efficaces** pour protéger votre culture !""",
        "causes": None  # Pas de causes affichées pour un cacao sain
    }
}

# 📌 Fonction pour prédire la maladie du cacao avec TFLite
def predict_cacao_disease_tflite(img_path):
    if not os.path.exists(img_path):
        print("❌ Erreur : L'image spécifiée n'existe pas.")
        return

    # Charger et prétraiter l'image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    # Exécuter l'inférence avec TFLite
    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])
    
    # Obtenir la classe prédite
    class_idx = np.argmax(predictions)
    class_names = ["black_pod_rot", "healthy", "pod_borer"]
    predicted_class = class_names[class_idx]
    
    # 📌 Afficher les résultats
    description = disease_info[predicted_class]["description"]
    causes = disease_info[predicted_class]["causes"]
    print(f"🔍 Maladie détectée : `{predicted_class}`\n")
    print(f"📖 Description :\n{description}\n")
    if causes:
        print(f"⚠️ Causes possibles :\n{causes}\n")
    else:
        print("✅ Continuez ainsi pour maintenir une culture saine ! 🌿🎉\n")

# 📌 Demander l'image à l'utilisateur
image_path = input("📸 Entrez le chemin de l'image du cacao : ")
predict_cacao_disease_tflite(image_path)


🔍 Maladie détectée : `black_pod_rot`

📖 Description :
🌿 *Pourriture Noire de la Cabosse (Black Pod Rot)* 🌿  
        La pourriture noire est une **maladie fongique** grave causée par *Phytophthora spp.*, qui attaque les cabosses de cacao.  
        Elle commence par l'apparition de **taches brunes foncées** qui s'élargissent progressivement, jusqu'à **recouvrir toute la cabosse** et la faire pourrir.  
        En l'absence de traitement, elle peut **détruire jusqu'à 40% de la production** d'une plantation.

⚠️ Causes possibles :
- **Humidité élevée et fortes pluies** favorisant la propagation du champignon.  
        - **Sol mal drainé** qui retient l'eau, créant un environnement idéal pour la maladie.  
        - **Spores fongiques transportées par le vent, l’eau ou les insectes**.  
        - **Absence de pratiques de lutte préventive** comme l'élagage et l'application de fongicides.

